# **IP3_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """You are the person in charge of packing in a large company. Your job is to skillfully pack items of various weights in a box with predetermined capacity. The objective is to use as few boxes as possible. There is a set of items and a set of boxes with an upper bound on the amount of boxes. Each of the items has a known weight. Each item is packed into one box and items are packed into the chosen boxes. Please formulate a mathematical optimization model for this problem.   """

In [6]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [8]:
print(response1.choices[0].message.content)

Parameters:
Let's denote:
- I as the set of items, where i ∈ I
- B as the set of boxes, where b ∈ B
- w[i] as the weight of item i
- c[b] as the capacity of box b
- M as the the upper limit on the number of boxes 

Decision variables:
Let's denote:
- x[i, b] as a binary decision variable which takes the value 1 if item i is packed in box b, and 0 otherwise.

Objective function:
We seek to minimize the number of boxes used. Thus the objective function is 
Min Σb∈B y[b]

where,
y[b] = 1, if any item is packed in box b (i.e., Σi∈I x[i,b] > 0)
y[b] = 0, otherwise 

Constraints:
1. Each item should be packed in one and only one box:
For every i ∈ I, Σb∈B x[i, b] =1 

2. Do not exceed the capacity of each box:
For every b ∈ B, Σi∈I w[i]*x[i,b] ≤ c[b]

3. Do not exceed the upper limit on the number of boxes:
Σb∈B y[b] ≤ M 

4. The decision variables x[i, b] and y[b] are binary, which means they should be either 0 or 1:
For every i ∈ I, for every b ∈ B, x[i, b] ∈ {0, 1}
For every b ∈ B, y[b] ∈

### **Generate Pyomo Code**

In [11]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [12]:
Markdown(response2.choices[0].message.content)

Here is a Python code for the above optimization problem using Pyomo package.

```python
from pyomo.environ import *

# Sets and parameters are defined - here I am using some random sample data

I = ['item1', 'item2', 'item3']  # Set of items
B = ['box1', 'box2', 'box3']  # Set of boxes
w = {'item1': 4, 'item2': 2, 'item3': 6}  # weight of items
c = {'box1': 10, 'box2': 8, 'box3': 6}  # capacity of boxes
M = 2  # maximum number of boxes

# Model
model = ConcreteModel()

# Variables
model.x = Var(I, B, domain=Binary)
model.y = Var(B, domain=Binary)

# Objective function
model.obj = Objective(expr=sum(model.y[b] for b in B), sense=minimize)

# Constraints
model.item_constraint = ConstraintList()
for i in I:
    model.item_constraint.add(sum(model.x[i, b] for b in B) == 1)

model.capacity_constraint = ConstraintList()
for b in B:
    model.capacity_constraint.add(sum(w[i]*model.x[i, b] for i in I) <= c[b])

model.Mbox_constraint = Constraint(expr=sum(model.y[b] for b in B) <= M)

model.box_indicator = ConstraintList()
for b in B:
    for i in I:
        model.box_indicator.add(model.x[i, b] <= model.y[b])

# Optimize
solver = SolverFactory('glpk')
solver.solve(model)

# Print the value of the variables at the optimum
for v in model.component_objects(Var, active=True):
    print ("Variable component object",v)
    for index in v:
        print ("  ",index, v[index].value)

```

In the above code `SolverFactory('glpk')` is used, which invokes the open-source GLPK solver. If you prefer to use a different solver, you can change it accordingly. It assumes that the said solver is install and configured in your system. You will need to follow the solver's specific instructions to installed and configure it.

### **Run the code Generated by GPT4**

In [14]:
from pyomo.environ import *

# Sets and parameters are defined - here I am using some random sample data

I = ['item1', 'item2', 'item3']  # Set of items
B = ['box1', 'box2', 'box3']  # Set of boxes
w = {'item1': 4, 'item2': 2, 'item3': 6}  # weight of items
c = {'box1': 10, 'box2': 8, 'box3': 6}  # capacity of boxes
M = 2  # maximum number of boxes

# Model
model = ConcreteModel()

# Variables
model.x = Var(I, B, domain=Binary)
model.y = Var(B, domain=Binary)

# Objective function
model.obj = Objective(expr=sum(model.y[b] for b in B), sense=minimize)

# Constraints
model.item_constraint = ConstraintList()
for i in I:
    model.item_constraint.add(sum(model.x[i, b] for b in B) == 1)

model.capacity_constraint = ConstraintList()
for b in B:
    model.capacity_constraint.add(sum(w[i]*model.x[i, b] for i in I) <= c[b])

model.Mbox_constraint = Constraint(expr=sum(model.y[b] for b in B) <= M)

model.box_indicator = ConstraintList()
for b in B:
    for i in I:
        model.box_indicator.add(model.x[i, b] <= model.y[b])

# Optimize
solver = SolverFactory('glpk')
solver.solve(model)

# Print the value of the variables at the optimum
for v in model.component_objects(Var, active=True):
    print ("Variable component object",v)
    for index in v:
        print ("  ",index, v[index].value)

Variable component object x
   ('item1', 'box1') 1.0
   ('item1', 'box2') 0.0
   ('item1', 'box3') 0.0
   ('item2', 'box1') 0.0
   ('item2', 'box2') 1.0
   ('item2', 'box3') 0.0
   ('item3', 'box1') 0.0
   ('item3', 'box2') 1.0
   ('item3', 'box3') 0.0
Variable component object y
   box1 1.0
   box2 1.0
   box3 0.0


In [18]:
from pyomo.environ import *

# Data inputted by human
I = list(range(1,25))     # Available items
B = list(range(1,16))  # Available boxes
w = {1: 2,2: 2,3: 2,4: 2,5: 3,6: 3,7:4,8: 4,9: 4,10: 4,11: 4,12:4,13: 5,14:5,15: 5,16: 5,17: 5,18: 5,19: 6,20:6,21: 7,22: 7,23: 8, 24:8} # Weight of each item
M = 15               # Maximum number of boxes
c = {1: 9,2: 9,3: 9,4: 9,5: 9,6: 9,7:9,8: 9,9: 9,10: 9,11: 9,12:9,13: 9,14:9,15: 9}  # capacity of boxes


# Model
model = ConcreteModel()

# Variables
model.x = Var(I, B, domain=Binary)
model.y = Var(B, domain=Binary)

# Objective function
model.obj = Objective(expr=sum(model.y[b] for b in B), sense=minimize)

# Constraints
model.item_constraint = ConstraintList()
for i in I:
    model.item_constraint.add(sum(model.x[i, b] for b in B) == 1)

model.capacity_constraint = ConstraintList()
for b in B:
    model.capacity_constraint.add(sum(w[i]*model.x[i, b] for i in I) <= c[b])

model.Mbox_constraint = Constraint(expr=sum(model.y[b] for b in B) <= M)

model.box_indicator = ConstraintList()
for b in B:
    for i in I:
        model.box_indicator.add(model.x[i, b] <= model.y[b])

# Optimize
solver = SolverFactory('glpk')
solver.solve(model)

# Print the value of the variables at the optimum
print(model.obj())

KeyboardInterrupt: 

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**